In [1]:
import os
import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras import utils
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from sklearn.model_selection import train_test_split

In [2]:
dataset_train = './dataset/asl_alphabet_train'
dataset_test = './dataset/asl_alphabet_test'
model_save = './model.hdf5'

In [3]:
def load_date(dir):
    imgs=[]
    labels=[]
    list_dir=os.listdir(dir)
    for label in range(len(list_dir)):
        print("Loading images form ", list_dir[label])
        for img_name in os.listdir(dir+"/"+list_dir[label]):
            img=cv.imread(dir+"/"+list_dir[label]+img_name)
            img+cv.resize(img, (48,48))
            imgs.append(img)
            labels.append(label)

X, Y = load_date(dataset_train)

Loading images form  A


error: OpenCV(4.8.0) D:\a\opencv-python\opencv-python\opencv\modules\imgproc\src\resize.cpp:4062: error: (-215:Assertion failed) !ssize.empty() in function 'cv::resize'


In [ ]:
X=np.array(X)
normalisedX=X.astype('float32')/255
label_encode=utils.to_categorical(Y)
x_train, x_test, y_train, y_test=train_test_split(normalisedX, label_encode, test_size=0.2)

In [ ]:
classes = 29
batch = 32
epochs = 15
learning_rate = 0.001

In [ ]:
model = Sequential()

model.add(Conv2D(64, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(128, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())
model.add(Dropout(0.2))

model.add(Conv2D(256, (3, 3), padding='same', input_shape=(48, 48, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(1024, activation='relu'))
model.add(Dense(classes, activation='softmax'))

In [ ]:
adam = Adam(lr=learning_rate)
model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, batch_size=batch, epochs=epochs, validation_split=0.2, shuffle = True, verbose=1)

In [ ]:
model.save(model_save)